Install OpenAI Gym.

In [1]:
!pip install gym
!pip install pyglet

You should consider upgrading via the 'c:\python38\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\python38\python.exe -m pip install --upgrade pip' command.


Tell OpenAI Gym Which environment we want to replicate

In [2]:
import gym
import random
import numpy as np
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 

Instructions for updating:
non-resource variables are not supported in the long term


In [3]:
env_name = "CartPole-v1"
env = gym.make(env_name)

In [4]:
class QNetwork():
    def __init__(self, state_dim, action_size):
        self.state_in = tf.placeholder(tf.float32, shape=[None, *state_dim])
        self.action_in = tf.placeholder(tf.int32, shape=[None])
        self.q_target_in = tf.placeholder(tf.float32, shape=[None])
        action_one_hot = tf.one_hot(self.action_in, depth=action_size)

        self.hidden1 = tf.layers.dense(self.state_in, 100, activation=tf.nn.relu)
        self.q_state = tf.layers.dense(self.hidden1, action_size, activation=None)
        self.q_state_action = tf.reduce_sum(
            tf.multiply(self.q_state, action_one_hot), axis = 1
        )
        

        self.loss = tf.reduce_mean(
            tf.square(
                self.q_state_action - self.q_target_in
            )
        )

        self.optimiser = tf.train.AdamOptimizer(0.001).minimize(self.loss)

    def update_model (self, session, state, action, q_target):
        feed = {self.state_in:state, self.action_in: action, self.q_target_in: q_target}
        session.run(self.optimiser, feed_dict=feed)

    def get_q_state(self, session, state):
        q_state = session.run(self.q_state, feed_dict={self.state_in: state})
        return q_state


In [5]:
class DQNAgent():
    def __init__(self, env):
        self.state_dim = env.observation_space.shape
        self.action_size = env.action_space.n
        
        self.q_network = QNetwork(self.state_dim, self.action_size)
        self.gamma = 0.97
        self.eps = 1

        self.sess = tf.Session()
        self.sess.run(tf.global_variables_initializer())
    
    def get_action(self, state):
        q_state = self.q_network.get_q_state(self.sess, [state])
        action_greedy = np.argmax(q_state)
        action_random = np.random.randint(self.action_size)

        action = action_random if random.random() < self.eps else action_greedy


        return action

    def train(self, state, action, next_state, reward, done):
        q_next_state = self.q_network.get_q_state(self.sess, [next_state])
        q_next_state = (1-done) * q_next_state
        q_target = reward + self.gamma * np.max(q_next_state)
        self.q_network.update_model(self.sess, [state], [action], [q_target])

        if done: self.eps = max (0.1, 0.99*self.eps)

    def __del__(self):
        self.sess.close()

In [6]:
agent = DQNAgent(env)
number_of_eps = 100

for ep in range(number_of_eps):
    state = env.reset()
    total_reward = 0
    done = False
    while not done:
        action = agent.get_action(state)
        next_state, reward, done, info = env.step(action)
        agent.train(state, action, next_state, reward, done)
        env.render()
        total_reward += reward
        state = next_state
    print("Episode: {}, total_reward: {:2f})".format(ep, total_reward))
    
    
env.close()


Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.


NameError: name 'action_random' is not defined